In [2]:
from dotenv import load_dotenv
import os, json
import requests

load_dotenv()
api_key = os.getenv("API_KEY", None)
project_id = os.getenv("PROJECT_ID", None)
ibm_cloud_iam_url = os.getenv("IAM_IBM_CLOUD_URL", None)

creds = {
    "url"    : "https://us-south.ml.cloud.ibm.com",
    "apikey" : api_key
}

params = {
    "decoding_method":"greedy",
    "max_new_tokens":3000,
    "min_new_tokens":1,
    # "temperature":0.1,
    "top_k":50,
    "top_p":1,
     "repetition_penalty": 1.1,
    "stop_sequences":["```","<|user|>","<|assistant|>"],
}

In [3]:
# Prepare the payload and headers
payload = {
    "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
    "apikey": api_key
}
headers = {
    'Content-Type': "application/x-www-form-urlencoded"
}

# Make a POST request while ignoring SSL certificate verification
try:
    response = requests.post(f"https://{ibm_cloud_iam_url}/identity/token", data=payload, headers=headers, verify=False)
    
    # Check if the request was successful
    response.raise_for_status()

    # Parse the JSON response
    decoded_json = response.json()
    access_token = decoded_json["access_token"]
    # print(f"Access Token: {access_token}")
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'iam.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [4]:
import requests

GRANITE_3B_CODE_INSTRUCT = "ibm/granite-3b-code-instruct"
GRANITE_8B_CODE_INSTRUCT = "ibm/granite-8b-code-instruct"
GRANITE_20B_CODE_INSTRUCT = "ibm/granite-20b-code-instruct"
GRANITE_34B_CODE_INSTRUCT = "ibm/granite-34b-code-instruct"
GRANITE_3_8B_INSTRUCT = "ibm/granite-3-8b-instruct"

def watsonx_code_gen(prompt,model_id):

    url = "https://us-south.ml.cloud.ibm.com/ml/v1/text/generation_stream?version=2023-05-29"

    body = {
        "input": prompt,
        "parameters": params,
        "model_id": model_id,
        "project_id": project_id
    }

    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": f"Bearer {access_token}"
    }

    response = requests.post(
        url,
        headers=headers,
        json=body,
        stream=True
    )

    if response.status_code != 200:
        raise Exception("Non-200 response: " + str(response.text))

    answer = ""
    # Stream the response
    for line in response.iter_lines():
        if line:  # Ensure the line is not empty
            decoded_line = line.decode("utf-8").strip()
            
            # Check if the line starts with "data: "
            if decoded_line.startswith("data: "):
                json_data = decoded_line[len("data: "):]  # Remove the "data: " prefix
                
                try:
                    # Attempt to load the JSON data
                    data = json.loads(json_data)
                    generated_text = data.get("results", "")
                    answer += generated_text[0]["generated_text"]
                    # print(generated_text[0]["generated_text"],end="")  # Print or process the generated text as needed
                except json.JSONDecodeError:
                    print("Failed to decode JSON:", json_data)
                except Exception as e:
                    print("An error occurred:", e)

    return answer


In [5]:
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
import os
from dotenv import load_dotenv


load_dotenv()
api_key = os.getenv("API_KEY", None)
project_id = os.getenv("PROJECT_ID", None)

creds = {
    "url"    : "https://us-south.ml.cloud.ibm.com",
    "apikey" : api_key
}

params = {
    GenParams.DECODING_METHOD:"greedy",
    GenParams.MAX_NEW_TOKENS:4000,
    GenParams.MIN_NEW_TOKENS:1,
    # "temperature":0.1,
    GenParams.TOP_K:50,
    GenParams.TOP_P:1,
    GenParams.STOP_SEQUENCES:["```","[/INST]","<|user|>","<|endoftext|>","<|assistant|>"],
}

model = ModelInference(model_id="ibm/granite-3-8b-instruct", credentials=creds, params=params, project_id=project_id)

In [6]:
model_id = GRANITE_3_8B_INSTRUCT
language = "python"
documentframework = "sphinx"
testframework = "unittest"

In [7]:
prompt = f"""<|user|>
Please generate a sample {language} code:
-give more detail and complex logic.
-generate the code only.
-dont generate explanation.
<|assistant|>
```{language}
"""

sample_code = model.generate_text(prompt).replace("```","")
print(sample_code)

import random

class ComplexObject:
    def __init__(self, name, value):
        self.name = name
        self.value = value

    def __str__(self):
        return f"Object: {self.name}, Value: {self.value}"

def generate_complex_object(name, value):
    return ComplexObject(name, value)

def perform_complex_operation(obj1, obj2):
    if obj1.value > obj2.value:
        return obj1
    else:
        return obj2

def main():
    obj1 = generate_complex_object("Object1", random.randint(1, 100))
    obj2 = generate_complex_object("Object2", random.randint(1, 100))

    print(f"Object1: {obj1}")
    print(f"Object2: {obj2}")

    result = perform_complex_operation(obj1, obj2)
    print(f"Result: {result}")

if __name__ == "__main__":
    main()



In [8]:
sample_code = """import random

class ComplexObject:
    def __init__(self, name, value):
        self.name = name
        self.value = value

    def __str__(self):
        return f"Object: {self.name}, Value: {self.value}"

def generate_complex_object(name, value):
    return ComplexObject(name, value)

def perform_complex_operation(obj1, obj2):
    if obj1.value > obj2.value:
        return obj1
    else:
        return obj2

def main():
    obj1 = generate_complex_object("Object1", random.randint(1, 100))
    obj2 = generate_complex_object("Object2", random.randint(1, 100))

    print(f"Object1: {obj1}")
    print(f"Object2: {obj2}")

    result = perform_complex_operation(obj1, obj2)
    print(f"Result: {result}")

if __name__ == "__main__":
    main()
"""

bad_code = """import random

class ComplexObject:
    def __init__(self, name, value):
        self.name = name
        self.value = value

    def __str__(self):
        return f"Object: {self.na

def generate_complex_object(name, value):
    return ComplexObject(name, value)

def perform_complex_operation(obj1, obj2):
    if obj1.valu
    else:
        return obj2

def main():
    obj1 = generate_complex_ob
    print(f"Object2: {obj2}")

    result = perform_complex_operation(obj1, obj2)
    print(f"Result: {result}")

if __name__ == "__main__":
    main()

"""

gnerate explanation

In [10]:
prompt = f"""<|user|>
Please provide a detailed explanation for the following {language} code:
-ensure have new line after every full stop.
-dont generate example.

###
code: `{sample_code}`
<|assistant|>
explanation:
"""

answer = model.generate_text(prompt)
print(answer)


This Python code defines a class named `ComplexObject` that represents an object with a name and a value. The `__str__` method is overridden to provide a custom string representation of the object.

The `generate_complex_object` function creates a new `ComplexObject` instance with the given name and a random value between 1 and 100.

The `perform_complex_operation` function takes two `ComplexObject` instances as arguments and returns the one with the higher value.

The `main` function generates two random `ComplexObject` instances, prints their details, and then calls `perform_complex_operation` to find and print the object with the higher value.

The `if __name__ == "__main__":` line ensures that the `main` function is called when the script is run directly, but not when it is imported as a module.

The code does not explicitly ensure a new line after every full stop, as this is handled by the Python interpreter's default behavior.


generate javadoc

In [21]:
prompt = f"""<|user|>
generate {documentframework} document for the {language} provided.
-generate the {language} code only.
-generate the {documentframework} inline documentation.
{language}: `{sample_code}`
<|assistant|>
```{language}"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


# ComplexObject.rst
ComplexObject Class

.. automodule:: complex_object
    :members:

.. autoclass:: ComplexObject
    :member-order: bysource

    .. automethod:: __init__
    .. automethod:: __str__

generate_complex_object Function

.. autofunction:: generate_complex_object

perform_complex_operation Function

.. autofunction:: perform_complex_operation

main Function

.. autofunction:: main



score, review and improve

In [13]:
prompt = f"""<|user|>
you are {language} expert. you need to review the following {language} code. 
-please understand the {language} code provided in detail.
-please review the code in completeness, logic, performance and quality.
-make it more object oriented.
-give a score out of 10 for the quality of code.
-please provide constructive suggestion in detail.
-generate the review and suggestion only.
{language} code: `{sample_code}`
-DONT generate the improve code directly.
-dont generate code.
<|assistant|>
score and review:"""

suggestion = model.generate_text(prompt)
print(suggestion)



The provided Python code is a simple script that creates two complex objects, compares their values, and returns the object with the higher value. Here's a detailed review of the code:

1. Completeness: The code is complete and fulfills its intended purpose. It defines a class for complex objects, generates two objects with random values, and performs a comparison operation.

2. Logic: The logic of the code is clear and easy to understand. The `ComplexObject` class is defined with a name and value attribute, and the `generate_complex_object` function creates instances of this class. The `perform_complex_operation` function compares the values of two objects and returns the one with the higher value.

3. Performance: The performance of the code is acceptable for its simplicity. However, the use of the `random.randint` function in the `main` function could be optimized by generating the random values only once and storing them in variables.

4. Quality: The code quality is good, but th

In [14]:
prompt = f"""<|user|>
improve the {language} provided base on the review provided.
-understand the review suggestion in detail.
-adopt the review suggestion if applicable.
-dont generate explanation.
###
original {language} code: `{sample_code}`
review suggestion: `{suggestion}`
<|assistant|>
improved {language}
```{language}"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


import random

class ComplexObject:
    def __init__(self, name, value):
        self.name = name
        self.value = value

    def __str__(self):
        return f"Object: {self.name}, Value: {self.value}"

    def compare_value(self, other):
        if self.value > other.value:
            return self
        else:
            return other

def generate_complex_object(name, value):
    if not isinstance(name, str) or not isinstance(value, int) or value <= 0:
        raise ValueError("Invalid name or value")
    return ComplexObject(name, value)

def main():
    obj1 = generate_complex_object("Object1", random.randint(1, 100))
    obj2 = generate_complex_object("Object2", random.randint(1, 100))

    print(f"Object1: {obj1}")
    print(f"Object2: {obj2}")

    result = obj1.compare_value(obj2)
    print(f"Result: {result}")

if __name__ == "__main__":
    main()



fix bad code

In [15]:
prompt = f"""<|user|>
fix the {language} provided.
-understand the review suggestion in detail.
-identify if any problem.
-fix the problem if any.
-add explanation as inline comment.

###
{language}: `{bad_code}`
<|assistant|>
fixed version with inline comment at the point it fix.
```{language}"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


import random

class ComplexObject:
    def __init__(self, name, value):
        self.name = name
        self.value = value

    def __str__(self):
        return f"Object: {self.name}, Value: {self.value}"

def generate_complex_object(name, value):
    return ComplexObject(name, value)

def perform_complex_operation(obj1, obj2):
    # check if obj1.value is greater than obj2.value
    if obj1.value > obj2.value:
        return obj1
    else:
        return obj2

def main():
    obj1 = generate_complex_object("Object1", random.randint(1, 100))
    obj2 = generate_complex_object("Object2", random.randint(1, 100))
    print(f"Object1: {obj1}")
    print(f"Object2: {obj2}")

    result = perform_complex_operation(obj1, obj2)
    print(f"Result: {result}")

if __name__ == "__main__":
    main()



generate diagram

In [16]:
prompt = f"""<|user|>
generate diagram for the {language} code provided.
-understand the review suggestion in detail.
-use simple name for node.
-generate in mermaid.
-dont repeating.
-dont generation explanation.
{language}: `{sample_code}`
<|assistant|>
```mermaid"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


graph TD
    A[Start] --> B[Generate Complex Object 1]
    B --> C[Generate Complex Object 2]
    C --> D[Perform Complex Operation]
    D --> E[Print Result]
    E --> F[End]
    B -->|Name| G[Object1]
    B -->|Value| H[Random Integer 1-100]
    C -->|Name| I[Object2]
    C -->|Value| J[Random Integer 1-100]
    D -->|Result| K[Object with Higher Value]
    E -->|Result| L[Object: K, Value: K.value]



conversion

In [20]:
target_language = "cpp"

prompt = f"""<|user|>
rewrite the {language} code provided to {target_language}.
-understand the code provided in detail.
-understand how to map technology and syntax to the target {target_language} langauge.
-dont generation explanation.
{language}: `{sample_code}`
<|assistant|>
```{target_language}"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


#include <iostream>
#include <cstdlib>
#include <ctime>

class ComplexObject {
public:
    std::string name;
    int value;

    ComplexObject(std::string name, int value) : name(name), value(value) {}

    friend std::ostream& operator<<(std::ostream& os, const ComplexObject& obj) {
        os << "Object: " << obj.name << ", Value: " << obj.value;
        return os;
    }
};

ComplexObject generate_complex_object(std::string name, int value) {
    return ComplexObject(name, value);
}

ComplexObject perform_complex_operation(const ComplexObject& obj1, const ComplexObject& obj2) {
    return obj1.value > obj2.value ? obj1 : obj2;
}

int main() {
    std::srand(std::time(nullptr));
    ComplexObject obj1 = generate_complex_object("Object1", std::rand() % 100 + 1);
    ComplexObject obj2 = generate_complex_object("Object2", std::rand() % 100 + 1);

    std::cout << "Object1: " << obj1 << std::endl;
    std::cout << "Object2: " << obj2 << std::endl;

    ComplexObject result = perform_com

generate api definition

In [18]:
prompt = f"""<|user|>
generate a API definition in swagger for the {language} code provided.
-dont generate explanation.
{language}: `{sample_code}`
<|assistant|>
```yaml
"""

answer = model.generate_text(prompt).replace("```","")
print(answer)

openapi: 3.0.0
info:
  title: Complex Object API
  version: 1.0.0
servers:
  - url: http://api.example.com/complex-object
paths:
  /generate_complex_object:
    post:
      summary: Generate a complex object
      requestBody:
        required: true
        content:
          application/json:
            schema:
              $ref: '#/components/schemas/ComplexObject'
      responses:
        '200':
          description: A complex object
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/ComplexObject'
  /perform_complex_operation:
    post:
      summary: Perform a complex operation
      requestBody:
        required: true
        content:
          application/json:
            schema:
              type: array
              items:
                $ref: '#/components/schemas/ComplexObject'
      responses:
        '200':
          description: The result of the complex operation
          content:
            applicat

generate test case

In [19]:
prompt = f"""<|user|>
generate Test Cases in {testframework} for the {language} code provided.
-focus on boundary cases.
-focus on exception handling
-generate the test only.
{language}: `{sample_code}`
<|assistant|>
```{language}"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


import unittest
from unittest.mock import patch
from your_module import ComplexObject, generate_complex_object, perform_complex_operation

class TestComplexObject(unittest.TestCase):
    def test_init(self):
        obj = ComplexObject("test", 123)
        self.assertEqual(obj.name, "test")
        self.assertEqual(obj.value, 123)

    def test_str(self):
        obj = ComplexObject("test", 123)
        self.assertEqual(str(obj), "Object: test, Value: 123")

class TestGenerateComplexObject(unittest.TestCase):
    def test_generate_complex_object(self):
        obj = generate_complex_object("test", 123)
        self.assertIsInstance(obj, ComplexObject)
        self.assertEqual(obj.name, "test")
        self.assertEqual(obj.value, 123)

class TestPerformComplexOperation(unittest.TestCase):
    @patch('random.randint')
    def test_perform_complex_operation(self, mock_randint):
        mock_randint.return_value = 123
        obj1 = ComplexObject("Object1", 123)
        obj2 = ComplexObje